In [89]:
import pandas as pd
import plotly_express as px
import numpy as np
from scipy import stats
import plotly.graph_objs as go
import plotly.io as pio
import io

# Data Formatting and cleaning

In [90]:
df = pd.read_excel('Edge Coupling .xlsx', skiprows=3)

In [91]:
df = df.iloc[:, :-1]

In [92]:
df

,Coupling length(um),Input Position,Output Position,Power (mW),Error (mW)
0,15,8,8,1.142000,0.013000
1,15,8,9,0.035400,0.000270
2,15,8,10,0.001189,0.000010
3,15,8,11,0.000058,0.000001
4,20,15,15,1.153000,0.007000
5,20,15,16,0.051200,0.000500
6,20,15,17,0.002570,0.000040
7,20,15,18,0.000133,0.000002


In [93]:
input_power = 28.61     # mW
input_power_error = 0.08    # mW

## Adding Log data column

In [94]:
df['log_fractional_power'] = np.log10(df['Power (mW)']/input_power)
df['loss'] = input_power - df['Power (mW)']
df['log_loss'] = np.log10(df['loss']/input_power)

In [95]:
df['Relative Position'] = df['Output Position'] - df['Input Position'] + 1

In [96]:
df

,Coupling length(um),Input Position,Output Position,Power (mW),Error (mW),log_fractional_power,loss,log_loss,Relative Position
0,15,8,8,1.142000,0.013000,-1.398852,27.468000,-1.769082e-02,1
1,15,8,9,0.035400,0.000270,-2.907515,28.574600,-5.376981e-04,2
2,15,8,10,0.001189,0.000010,-4.381336,28.608811,-1.804917e-05,3
3,15,8,11,0.000058,0.000001,-5.693090,28.609942,-8.804301e-07,4
4,20,15,15,1.153000,0.007000,-1.394689,27.457000,-1.786477e-02,1
5,20,15,16,0.051200,0.000500,-2.747248,28.558800,-7.779028e-04,2
6,20,15,17,0.002570,0.000040,-4.046585,28.607430,-3.901387e-05,3
7,20,15,18,0.000133,0.000002,-5.332666,28.609867,-2.018920e-06,4


## Sepereating data according to coupler length

In [97]:
x_15 = df.loc[df["Coupling length(um)"] == 15, "Relative Position"]
y_15 = df.loc[df["Coupling length(um)"] == 15, "log_fractional_power"]

x_20 = df.loc[df["Coupling length(um)"] == 20, "Relative Position"]
y_20 = df.loc[df["Coupling length(um)"] == 20, "log_fractional_power"]

## Regression analysis and construction of best fit lines

In [98]:
slope_15, intercept_15, r_value_15, p_value_15, std_err_15 = stats.linregress(x_15, y_15)
line_15 = slope_15 * x_15 + intercept_15
eq_15 = "y = " + str(round(slope_15, 2)) + "x + " + str(round(intercept_15, 2))

slope_20, intercept_20, r_value_20, p_value_20, std_err_20 = stats.linregress(x_20, y_20)
line_20 = slope_20 * x_20 + intercept_20
eq_20 = "y = " + str(round(slope_20, 2)) + "x + " + str(round(intercept_20, 2))



In [99]:
df['Coupling length(um)'] = df['Coupling length(um)'].astype(str)
df['Relative Position'] = df['Relative Position'].astype(int)

# Plotting points and lines via plotly

In [100]:
fig = px.scatter(df, x='Relative Position', y='log_fractional_power', color='Coupling length(um)')
fig.update_layout(title_text="Plot of output power vs number of coupler interactions", title_font_size=18, title_x=0.5, template="none", legend_title="Coupling Length (uM)", margin_b=50, margin_t=60)
fig.update_xaxes(showgrid=False, title_text="Coupler Interactions", showline=True, linewidth=2, linecolor='black', mirror=True, ticks="inside", tickwidth=2, tickcolor='black', ticklen=10, title_standoff=10)
fig.update_yaxes(title_text="log(P/Po)", showline=True, linewidth=2, linecolor='black', mirror=True, ticks="inside", tickwidth=2, tickcolor='black', ticklen=10, title_standoff=10)
fig.add_trace(go.Scatter(x=x_15, y=line_15, mode='lines', name='Regression Line for 15uM', line_color='blue'))
fig.add_trace(go.Scatter(x=x_20, y=line_20, mode='lines', name='Regression Line for 20uM', line_color='red'))
fig.add_annotation(x=max(x_15), y=min(y_15), xanchor='right', yanchor='bottom', text=eq_15, font=dict(family="Arial", size=16, color="black"))
fig.add_annotation(x=max(x_20), y=min(y_20), xanchor='right', yanchor='bottom', text=eq_20, font=dict(family="Arial", size=16, color="black"))
fig.show()

## Saving image and regression results

In [101]:
# pio.write_image(fig, 'figure.svg')

In [102]:
regression_results = {
    "Coupler length(uM)": [15,20],
    "slope": [slope_15, slope_20],
    "intercept": [intercept_15, intercept_20],
    "r_value": [r_value_15, r_value_20],
    "std_err": [std_err_15, std_err_20],
    "equation": [eq_15, eq_20]
}

# Create a pandas dataframe from the dictionary
df_regression_results = pd.DataFrame(regression_results)

# Display the dataframe
print(df_regression_results)

   Coupler length(uM)     slope  intercept   r_value   std_err   
0                  15 -1.435654  -0.006064 -0.999491  0.032408  \
1                  20 -1.311327  -0.101979 -0.999931  0.010884   

             equation  
0  y = -1.44x + -0.01  
1   y = -1.31x + -0.1  


In [103]:
metadata_2 = f"Input Power (mW) = {input_power} +/- {input_power_error}"
metadata_1 = "R42-D103"
df_str = io.StringIO()
df_regression_results.to_csv(df_str, header=True, index=False, sep='\t')
df_with_metadata = metadata_1 + "\n" + metadata_2 + "\n" + df_str.getvalue()

with open("T5-R42-D103", "w") as f:
    f.write(df_with_metadata)